1. *Самостоятельно повторить tfidf (документация https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [1]:
import pandas as pd

Наши новости

In [2]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR" -O articles.csv && rm -rf /tmp/cookies.txt 
!wget 'https://drive.google.com/uc?export=download&id=1Q97K9eGrvpbS4ut5CphZa--gJDRqQp2a' -O users_articles.csv

"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
"id" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [3]:
news = pd.read_csv("./Lection2/materials.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


Загрузим пользователей и списки последних прочитанных новостей

In [4]:
users = pd.read_csv("./Lection2/users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


Итак, нам нужно получить векторные представления пользователей на основе прочитанным ими новостей и самих новостей

### 1. Получаем векторные представления новостей

In [5]:
!pip install razdel pymorphy2 gensim nltk


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
# предобработка текстов
import re
import numpy as np
from gensim.corpora.dictionary import Dictionary
from razdel import tokenize  # сегментация русскоязычного текста на токены и предложения https://github.com/natasha/razdel
import pymorphy2  # Морфологический анализатор

In [7]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fitz1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
stopword_ru = stopwords.words('russian')
print(len(stopword_ru))

151


In [9]:
!wget 'https://drive.google.com/uc?export=download&id=17wVn5YPpMjHToctGgff_KfSeWcIIlf7c' -O stopwords.txt

"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
"id" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [10]:
with open('./Lection2/stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
    
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [11]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    text = re.sub('n', ' ', text)
    
    return text

cache = {}
morph = pymorphy2.MorphAnalyzer()

def lemmatization(text):    
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист лемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w) > 1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords = [i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [12]:
from tqdm import tqdm
tqdm.pandas()

# Запускаем очистку текста. Будет долго...
news['title'] = news['title'].progress_apply(lambda x: clean_text(x))

  0%|                                                                                                                                                                                                           | 0/27000 [00:00<?, ?it/s]C:\Users\fitz1\AppData\Local\Temp\ipykernel_3180\4083466619.py:14: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27000/27000 [00:18<00:00, 1497.03it/s]


In [13]:
news['title'].iloc[:10]

0    заместитель председателя правительства рф серг...
1    матч  финала кубка россии по футболу был приос...
2    форвард авангарда томаш заборский прокомментир...
3    главный тренер кубани юрий красножан прокоммен...
4    решением попечительского совета владивостокско...
5    ио главного тренера вячеслав буцаев прокоммент...
6    запорожский металлург дома потерпел разгромное...
7    сборная сша одержала победу над австрией со сч...
8    бывший защитник сборной россии дарюс каспарайт...
9    полузащитник цска зоран тошич после победы над...
Name: title, dtype: object

In [14]:
# Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].progress_apply(lambda x: lemmatization(x))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27000/27000 [02:15<00:00, 199.71it/s]


А теперь в 3 строчки обучим нашу модель

In [15]:
# сформируем список наших текстов
texts = list(news['title'].values)

# Создадим корпус из списка с текстами
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

Запускаем обучение

In [16]:
N_topic = 20  # можно было менять

In [17]:
from gensim.models import LdaModel

# Обучаем модель на корпусе
lda = LdaModel(common_corpus, num_topics=N_topic, id2word=common_dictionary, passes=2)  # можно было менять

In [18]:
from gensim.test.utils import datapath

# Сохраняем модель на диск
temp_file = datapath("model.lda")
lda.save(temp_file)

In [19]:
# Загружаем обученную модель с диска
lda = LdaModel.load(temp_file)

In [20]:
# Создаем новый корпус документов, которые раньше не видели
other_texts = list(news['title'].iloc[:3])
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'свой', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'провести', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'работа', 'сказать', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'мочь', 'играть', 'ещё', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'это', 'хороший']


[(1, 0.18597247),
 (5, 0.1305935),
 (8, 0.044049595),
 (12, 0.17404556),
 (16, 0.30734137),
 (17, 0.14196357)]

Обучили модель. Теперь 2 вопроса:

1. как выглядят наши темы
2. как получить для документа вектор значений (вероятности принадлежности каждой теме)

In [21]:
x = lda.show_topics(num_topics=N_topic, num_words=7, formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

# Печатаем только слова
for topic, words in topics_words:
    print(f"topic_{topic}: " + " ".join(words))

topic_0: год рубль млн млрд рост тыс компания
topic_1: исследование год погибнуть выяснить день смерть пациент
topic_2: вуз лекарство кг медведь олимпийский спорт соревнование
topic_3: взрыв экипаж топливо пилот производить секунда диск
topic_4: дело суд год уголовный который убийство сотрудник
topic_5: германия форум франция парламент великобритания египет сообщество
topic_6: турция турецкий активность авария доклад лаборатория автомобиль
topic_7: который человек это год ребёнок земля свой
topic_8: ракета журнал пенсия год снижение двигатель налог
topic_9: украина россия украинский президент российский владимир путин
topic_10: газ фестиваль мероприятие пройти сайт программа организатор
topic_11: который это год свой также банк мочь
topic_12: это год россия который российский проект новый
topic_13: расчёт энергия восток обращение теория констатировать тепло
topic_14: сша американский это военный россия который статья
topic_15: университет студент озеро египетский круглый химия железный

Очень неплохо - большинство тем вполне можно описать о чем они

Давайте напишем функцию, которая будет нам возвращать векторное представление новости

In [22]:
def get_lda_vector(lda, text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]

    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(N_topic):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [23]:
get_lda_vector(lda, news['title'].iloc[0])

array([0.        , 0.        , 0.05350644, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.10321938,
       0.        , 0.70053887, 0.07853249, 0.        , 0.        ,
       0.        , 0.05731726, 0.        , 0.        , 0.        ])

In [24]:
topic_matrix = pd.DataFrame([get_lda_vector(lda, text) for text in news['title'].values])
topic_matrix.columns = [f'topic_{i}' for i in range(N_topic)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+[f'topic_{i}' for i in range(N_topic)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,6,0.0,0.000000,0.053507,0.000000,0.00000,0.000000,0.0,0.0,0.000000,...,0.0,0.700580,0.078539,0.000000,0.0,0.0,0.057318,0.000000,0.0,0.0
1,4896,0.0,0.000000,0.056485,0.214329,0.10309,0.000000,0.0,0.0,0.000000,...,0.0,0.250365,0.000000,0.000000,0.0,0.0,0.356966,0.000000,0.0,0.0
2,4897,0.0,0.185903,0.000000,0.000000,0.00000,0.130582,0.0,0.0,0.044064,...,0.0,0.000000,0.173749,0.000000,0.0,0.0,0.307202,0.142467,0.0,0.0
3,4898,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.099586,...,0.0,0.000000,0.232165,0.015461,0.0,0.0,0.205652,0.379624,0.0,0.0
4,4899,0.0,0.158712,0.000000,0.000000,0.00000,0.232723,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0


Прекрасно, мы получили вектора наших новостей! И даже умеем интерпретировать получившиеся темы.

Можно двигаться далее

### Следующий шаг - векторные представления пользователей

In [25]:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [26]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[[f'topic_{i}' for i in range(N_topic)]].values))

In [27]:
doc_dict[293672]

array([0.19014241, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.08686335,
       0.02896884, 0.        , 0.58046323, 0.02891507, 0.06908461,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [28]:
def get_user_embedding(user_articles_list, doc_dict, func):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = func(user_vector, axis=0)
    return user_vector

In [29]:
user_articles_list = users['articles'].iloc[33]

get_user_embedding(user_articles_list, doc_dict, np.mean)

array([0.0487423 , 0.020824  , 0.        , 0.00194777, 0.11769281,
       0.02191097, 0.        , 0.0988786 , 0.00906187, 0.15075414,
       0.        , 0.21875342, 0.11520825, 0.        , 0.11918096,
       0.        , 0.        , 0.04086915, 0.        , 0.02618209])

Теперь получим эмбединги для всех пользователей и проверим их качество на конкретной downstream-задаче

In [30]:
FUNC = np.mean

In [31]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,u105138,0.031690,0.050132,0.005612,0.027008,0.039007,0.000000,0.000000,0.203903,0.027479,...,0.004828,0.124176,0.181456,0.004819,0.047718,0.0,0.001673,0.041574,0.000000,0.003519
1,u108690,0.057628,0.019502,0.003230,0.010491,0.084325,0.005369,0.002534,0.104195,0.009170,...,0.000000,0.277434,0.058324,0.000000,0.052875,0.0,0.002702,0.111817,0.000000,0.005577
2,u108339,0.048882,0.082001,0.000000,0.000000,0.117549,0.019952,0.010138,0.176585,0.001831,...,0.007500,0.266076,0.051669,0.000000,0.042756,0.0,0.002601,0.051376,0.006396,0.002436


Датасет готов - можно попробовать обучить модель. Загрузим нашу разметку

In [32]:
!wget 'https://drive.google.com/uc?export=download&id=1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK' -O users_churn.csv

"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
"id" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [33]:
target = pd.read_csv("./Lection2/users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [34]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,churn
0,u105138,0.031690,0.050132,0.005612,0.027008,0.039007,0.000000,0.000000,0.203903,0.027479,...,0.124176,0.181456,0.004819,0.047718,0.0,0.001673,0.041574,0.000000,0.003519,0
1,u108690,0.057628,0.019502,0.003230,0.010491,0.084325,0.005369,0.002534,0.104195,0.009170,...,0.277434,0.058324,0.000000,0.052875,0.0,0.002702,0.111817,0.000000,0.005577,1
2,u108339,0.048882,0.082001,0.000000,0.000000,0.117549,0.019952,0.010138,0.176585,0.001831,...,0.266076,0.051669,0.000000,0.042756,0.0,0.002601,0.051376,0.006396,0.002436,1


In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [36]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [37]:
model = LogisticRegression()
# обучим 
model.fit(X_train, y_train)

LogisticRegression()

In [38]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([0.09089473, 0.02956611, 0.72819149, 0.23845027, 0.0418964 ,
       0.08759446, 0.17750616, 0.07012568, 0.03353378, 0.1459417 ])

In [39]:
metrics_df = pd.DataFrame(columns=['model', 'thresh', 'F-Score', 'Precision', 'Recall', 'ROC AUC'])
metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC


In [40]:
from sklearn.metrics import (f1_score, roc_auc_score, precision_score,
                             classification_report, precision_recall_curve, confusion_matrix)

In [41]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
eps = 1e-10
fscore = (2 * precision * recall) / (precision + recall + eps)

# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.28403982428003965, F-Score=0.785, Precision=0.746, Recall=0.829


In [42]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9764009535438106

In [43]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.28404,0.7853,0.746324,0.828571,0.976401


2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог)

In [44]:
FUNC = np.median

In [45]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,u105138,0.000000,0.019922,0.0,0.008473,0.037318,0.0,0.000000,0.259732,0.0,...,0.0,0.150083,0.023914,0.0,0.034542,0.0,0.0,0.019850,0.0,0.0
1,u108690,0.022184,0.016160,0.0,0.000000,0.038132,0.0,0.000000,0.107148,0.0,...,0.0,0.276409,0.036590,0.0,0.016570,0.0,0.0,0.087331,0.0,0.0
2,u108339,0.019680,0.062098,0.0,0.000000,0.110147,0.0,0.007853,0.158420,0.0,...,0.0,0.283058,0.053387,0.0,0.016890,0.0,0.0,0.046143,0.0,0.0


In [46]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,churn
0,u105138,0.000000,0.019922,0.0,0.008473,0.037318,0.0,0.000000,0.259732,0.0,...,0.150083,0.023914,0.0,0.034542,0.0,0.0,0.019850,0.0,0.0,0
1,u108690,0.022184,0.016160,0.0,0.000000,0.038132,0.0,0.000000,0.107148,0.0,...,0.276409,0.036590,0.0,0.016570,0.0,0.0,0.087331,0.0,0.0,1
2,u108339,0.019680,0.062098,0.0,0.000000,0.110147,0.0,0.007853,0.158420,0.0,...,0.283058,0.053387,0.0,0.016890,0.0,0.0,0.046143,0.0,0.0,1


In [47]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [48]:
model = LogisticRegression()
# обучим 
model.fit(X_train, y_train)

LogisticRegression()

In [49]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([0.18868905, 0.03427846, 0.56616233, 0.40884395, 0.06075144,
       0.03305979, 0.2053714 , 0.02148963, 0.07853603, 0.08298325])

In [50]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.30758263656472506, F-Score=0.775, Precision=0.783, Recall=0.767


In [51]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9726425954997384

In [52]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.284040,0.785300,0.746324,0.828571,0.976401
1,median,0.307583,0.775258,0.783333,0.767347,0.972643



3. Повторить п.2, но используя уже не медиану, а max


In [53]:
FUNC = np.max

In [54]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,u105138,0.190142,0.166739,0.033672,0.118408,0.080785,0.000000,0.000000,0.376774,0.082631,...,0.028969,0.252910,0.580463,0.028915,0.128148,0.0,0.010040,0.149123,0.000000,0.021113
1,u108690,0.155962,0.046794,0.019383,0.062943,0.379244,0.032211,0.015204,0.200831,0.035392,...,0.000000,0.411542,0.201916,0.000000,0.148599,0.0,0.016214,0.296569,0.000000,0.022602
2,u108339,0.133186,0.228134,0.000000,0.000000,0.233792,0.119709,0.026083,0.456167,0.010989,...,0.045002,0.385742,0.109509,0.000000,0.187974,0.0,0.015605,0.106219,0.038378,0.014614


In [55]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,churn
0,u105138,0.190142,0.166739,0.033672,0.118408,0.080785,0.000000,0.000000,0.376774,0.082631,...,0.252910,0.580463,0.028915,0.128148,0.0,0.010040,0.149123,0.000000,0.021113,0
1,u108690,0.155962,0.046794,0.019383,0.062943,0.379244,0.032211,0.015204,0.200831,0.035392,...,0.411542,0.201916,0.000000,0.148599,0.0,0.016214,0.296569,0.000000,0.022602,1
2,u108339,0.133186,0.228134,0.000000,0.000000,0.233792,0.119709,0.026083,0.456167,0.010989,...,0.385742,0.109509,0.000000,0.187974,0.0,0.015605,0.106219,0.038378,0.014614,1


In [56]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [57]:
model = LogisticRegression()
# обучим 
model.fit(X_train, y_train)

LogisticRegression()

In [58]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([0.04427919, 0.00209217, 0.95424305, 0.10825128, 0.02002307,
       0.01416081, 0.11213354, 0.00670797, 0.00308416, 0.10046343])

In [59]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.29079054158370876, F-Score=0.808, Precision=0.743, Recall=0.886


In [60]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9793546136403279

In [61]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.284040,0.785300,0.746324,0.828571,0.976401
1,median,0.307583,0.775258,0.783333,0.767347,0.972643
2,amax,0.290791,0.808194,0.743151,0.885714,0.979355


4. *Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (взяв список новостей пользователя)
	- подсказка 1: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал
	- подсказка 2: нужен именно idf, как вес.

In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [63]:
users['articles_str'] = users['articles'].apply(lambda x: x.replace('[','').replace(']', '').replace(',', ''))

users['articles_str'].iloc[0]

'293672 293328 293001 293622 293126 1852'

In [64]:
tfidf = TfidfVectorizer()
tfidf.fit(users['articles_str'])

TfidfVectorizer()

In [65]:
idf = pd.DataFrame({'article_id': tfidf.get_feature_names_out(),
                    'idf': tfidf.idf_})

idf

,article_id,idf
0,10,8.888710
1,100,7.907880
2,1000,8.041412
3,1001,8.888710
4,1002,8.888710
...,...,...
14776,995,8.377884
14777,996,8.195562
14778,997,8.601027
14779,998,9.294175


In [66]:
def get_user_embedding_idf(user_articles_list, doc_dict):
    user_articles_list = eval(user_articles_list)
    
    user_vector = np.zeros((len(user_articles_list), N_topic))
    for i, doc_id in enumerate(user_articles_list):
        try:
            weight = idf[idf['article_id'] == str(doc_id)]['idf'].values[0]
        except Exception as e:
            weight = 0
        user_vector[i] = doc_dict[doc_id] * weight

    user_vector = np.mean(user_vector, axis=0)
    return user_vector

In [67]:
from tqdm import tqdm
tqdm.pandas()

user_embeddings = pd.DataFrame([i for i in users['articles'].progress_apply(lambda x: get_user_embedding_idf(x, doc_dict))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8000/8000 [00:43<00:00, 183.40it/s]


,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,u105138,0.272570,0.422290,0.052160,0.234254,0.338813,0.000000,0.000000,1.751519,0.242836,...,0.041527,1.080973,1.543585,0.04145,0.419209,0.0,0.014392,0.354509,0.000000,0.030266
1,u108690,0.503179,0.170507,0.027785,0.093248,0.728243,0.041822,0.023551,0.898971,0.079246,...,0.000000,2.400612,0.483535,0.00000,0.440430,0.0,0.024020,0.968839,0.000000,0.047235
2,u108339,0.407326,0.692954,0.000000,0.000000,1.010224,0.155425,0.088426,1.535404,0.017022,...,0.062838,2.223114,0.440982,0.00000,0.361255,0.0,0.021316,0.429911,0.053588,0.020949


In [68]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,churn
0,u105138,0.272570,0.422290,0.052160,0.234254,0.338813,0.000000,0.000000,1.751519,0.242836,...,1.080973,1.543585,0.04145,0.419209,0.0,0.014392,0.354509,0.000000,0.030266,0
1,u108690,0.503179,0.170507,0.027785,0.093248,0.728243,0.041822,0.023551,0.898971,0.079246,...,2.400612,0.483535,0.00000,0.440430,0.0,0.024020,0.968839,0.000000,0.047235,1
2,u108339,0.407326,0.692954,0.000000,0.000000,1.010224,0.155425,0.088426,1.535404,0.017022,...,2.223114,0.440982,0.00000,0.361255,0.0,0.021316,0.429911,0.053588,0.020949,1


In [69]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [70]:
model = LogisticRegression()
# обучим 
model.fit(X_train, y_train)

c:\users\fitz1\desktop\python_ds\python\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [71]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([1.00842466e-02, 8.62124484e-07, 9.78493142e-01, 1.83544908e-01,
       8.03003283e-03, 5.36711768e-02, 1.01682874e-01, 5.41786148e-04,
       1.14660807e-07, 8.46226980e-02])

In [72]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.4296171275297086, F-Score=0.901, Precision=0.889, Recall=0.914


In [73]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9929367986510844

In [74]:
metrics_df = metrics_df.append({
    'model': 'idf_mean',
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)


5. Сформировать на выходе единую таблицу, сравнивающую качество 2/3 разных метода получения эмбедингов пользователей: median, max, idf_mean по метрикам roc_auc, precision, recall, f_score


In [75]:
metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.284040,0.785300,0.746324,0.828571,0.976401
1,median,0.307583,0.775258,0.783333,0.767347,0.972643
2,amax,0.290791,0.808194,0.743151,0.885714,0.979355
3,idf_mean,0.429617,0.901408,0.888889,0.914286,0.992937


6. Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

Способ с нахождением весов новостей с помощью idf из tfifdvectorizer оказался эффективнее, поскольку не все новости (документы) одинаково полезны для модели. idf Присваивает высокий вес более характерным и редким новостям, что позволяет построить более точные эмбеддинги пользователей.